# Basics

## Imports

In [1]:
import pandas as pd
import os
from pathlib import Path

## Load Data

In [2]:
input_file_foras = list(Path(os.path.join('data')).glob('PTSS_Data_Foras.xlsx'))[0]
input_file_synergy = list(Path(os.path.join('data')).glob('PTSS_Data_Synergy.xlsx'))[0]
fulltext_foras = list(Path(os.path.join('data')).glob('PTSS_Data_Foras_Fulltext.xlsx'))[0]
fulltext_synergy = list(Path(os.path.join('data')).glob('PTSS_Data_Synergy_Fulltext.xlsx'))[0]

print("Results based on file: ", input_file_foras, input_file_synergy, fulltext_foras, fulltext_synergy)

# Read the foras file and filter out the duplicates
foras_unfiltered = pd.read_excel(input_file_foras)
foras_filtered = foras_unfiltered[foras_unfiltered['filter_duplicate'] != 1]

# Read the other files
synergy = pd.read_excel(input_file_synergy)
fulltext_foras = pd.read_excel(fulltext_foras)
fulltext_synergy = pd.read_excel(fulltext_synergy)

# Print the number of rows in each file
print("Number of records in original FORAS file: ", foras_unfiltered.shape[0])
print("Number of records in FORAS after filtering duplicates: ", foras_filtered.shape[0])
print("Number of records in SYNERGY", synergy.shape[0])
print("Number of records in FORAS fulltext", fulltext_foras.shape[0])
print("Number of records in SYNERGY fulltext", fulltext_synergy.shape[0])

Results based on file:  data\PTSS_Data_Foras.xlsx data\PTSS_Data_Synergy.xlsx data\PTSS_Data_Foras_Fulltext.xlsx data\PTSS_Data_Synergy_Fulltext.xlsx
Number of records in original FORAS file:  10595
Number of records in FORAS after filtering duplicates:  10523
Number of records in SYNERGY 4544
Number of records in FORAS fulltext 260
Number of records in SYNERGY fulltext 47
